# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

In [4]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import lib
importlib.reload(lib)

In [6]:
from lib.lsp.basis_generator import bivariate_legendre, bivariate_chebyshev
from lib.lsp.learn_solution_path import learn_solution_path
from lib.lsp.loss_fn_lsp import allocation_cost_no_con
from lib.lsp.utils_lsp import get_sup_error_lsp_2d
from lib.fast_tensor_data_loader import FastTensorDataLoader

# 02 Instantiate dataset

In [7]:
# file path
decomp_cov_df = pd.read_csv('decomp_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [8]:
decomp_cov = np.array(decomp_cov_df)
mean = np.array(mean_df).squeeze()

In [9]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [207]:
# identity_matrix = np.eye(len(decomp_cov))

# decomp_cov = torch.tensor(identity_matrix, dtype=torch.float32)

In [10]:
decomp_cov

tensor([[ 3.8584,  4.6557,  4.0503,  4.7056,  3.3298,  3.4872,  3.2121,  2.7176,
          2.7919,  3.9016],
        [ 0.0000,  9.6246,  1.4167,  1.5061,  2.6207,  0.6535,  2.1278,  0.8926,
         -0.1126,  1.8297],
        [ 0.0000,  0.0000,  2.6853,  2.6613,  1.4872,  1.3278,  1.2272,  1.4394,
          0.4031,  2.2804],
        [ 0.0000,  0.0000,  0.0000,  6.7527, -0.6083,  0.5163, -0.4300, -0.2296,
         -0.1087,  0.6784],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  3.0578,  0.4449,  1.4199,  0.8157,
          0.1635,  0.2507],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9848,  0.2056,  0.2594,
         -0.0997,  0.5166],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2011,  0.7656,
          0.1802,  0.1505],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  2.6989,
         -0.2101,  0.1462],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          3.0994, -0.2774],
        [ 0.0000,  

In [11]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

In [12]:
lam_max_2d = [1, 1]
lam_min_2d = [0.2, 0]
input_dim = decomp_cov.shape[1]

In [13]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list_cov_001_100_100_no_con.csv')

# Display the DataFrame
truth

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.013926,-0.013637,-0.013352,-0.013070,-0.012791,-0.012515,-0.012242,-0.011973,-0.011707,-0.011445,...,-0.000112,-0.000088,-0.000068,-0.000050,-0.000034,-0.000022,-0.000012,-0.000006,-0.000001,0.0
1,-0.014002,-0.013711,-0.013424,-0.013140,-0.012859,-0.012582,-0.012308,-0.012038,-0.011770,-0.011506,...,-0.000112,-0.000089,-0.000068,-0.000050,-0.000035,-0.000022,-0.000012,-0.000006,-0.000001,0.0
2,-0.014078,-0.013786,-0.013497,-0.013211,-0.012929,-0.012650,-0.012375,-0.012103,-0.011834,-0.011568,...,-0.000113,-0.000089,-0.000068,-0.000050,-0.000035,-0.000022,-0.000013,-0.000006,-0.000001,0.0
3,-0.014155,-0.013861,-0.013571,-0.013284,-0.013000,-0.012719,-0.012442,-0.012169,-0.011898,-0.011631,...,-0.000113,-0.000090,-0.000069,-0.000050,-0.000035,-0.000022,-0.000013,-0.000006,-0.000001,0.0
4,-0.014234,-0.013938,-0.013646,-0.013357,-0.013071,-0.012789,-0.012511,-0.012235,-0.011963,-0.011695,...,-0.000114,-0.000090,-0.000069,-0.000051,-0.000035,-0.000023,-0.000013,-0.000006,-0.000001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.034586,-0.033755,-0.032940,-0.032141,-0.031358,-0.030589,-0.029835,-0.029096,-0.028371,-0.027660,...,-0.000249,-0.000197,-0.000151,-0.000111,-0.000077,-0.000049,-0.000028,-0.000012,-0.000003,0.0
96,-0.035320,-0.034465,-0.033626,-0.032804,-0.031998,-0.031208,-0.030434,-0.029675,-0.028930,-0.028200,...,-0.000253,-0.000200,-0.000153,-0.000112,-0.000078,-0.000050,-0.000028,-0.000012,-0.000003,0.0
97,-0.036098,-0.035216,-0.034352,-0.033505,-0.032676,-0.031863,-0.031066,-0.030285,-0.029520,-0.028770,...,-0.000257,-0.000203,-0.000155,-0.000114,-0.000079,-0.000051,-0.000029,-0.000013,-0.000003,0.0
98,-0.036925,-0.036015,-0.035123,-0.034249,-0.033394,-0.032556,-0.031735,-0.030931,-0.030143,-0.029372,...,-0.000261,-0.000206,-0.000158,-0.000116,-0.000080,-0.000051,-0.000029,-0.000013,-0.000003,0.0


In [14]:
true_losses = truth.to_numpy()

In [15]:
loss_fn = allocation_cost_no_con

# 03 Learn Solution Path with BFGS

In [16]:
phi_lam = bivariate_legendre
# phi_lam = bivariate_chebyshev

In [27]:
def thresh_basis(basis_dim):
    return -1

## num basis func = 4

In [33]:
basis_dim = 4
init_lr = .1

In [34]:
max_epochs = 1000

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                                               diminish=False, 
                                                               intercept=False, weighted_avg=False, thresh_basis=thresh_basis,
                                                               record_fctn=get_sup_error_lsp_2d,
                                                               record_frequency=1, distribution='uniform',
                                                               opt_method='lbfgs',
                                                               device=device, trace_frequency=1)
sup_err_history = np.array(sup_err_history)

In [ ]:
file_path = 'LSP_results_exact_fixed_basis_2d_no_con_basis_4.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=True)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

## num basis func = 9

In [41]:
basis_dim = 9
init_lr = .1

In [40]:
max_epochs = 1000

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                                               diminish=False, 
                                                               intercept=False, weighted_avg=False, thresh_basis=thresh_basis,
                                                               record_fctn=get_sup_error_lsp_2d,
                                                               record_frequency=1, distribution='uniform',
                                                               opt_method='lbfgs',
                                                               device=device, trace_frequency=1)
sup_err_history = np.array(sup_err_history)

In [ ]:
file_path = 'LSP_results_exact_fixed_basis_2d_no_con_basis_9.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=True)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

## num basis func = 16

In [51]:
basis_dim = 16
init_lr = .1

In [52]:
max_epochs = 1000

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                                               diminish=False, 
                                                               intercept=False, weighted_avg=False, thresh_basis=thresh_basis,
                                                               record_fctn=get_sup_error_lsp_2d,
                                                               record_frequency=1, distribution='uniform',
                                                               opt_method='lbfgs',
                                                               device=device, trace_frequency=1)
sup_err_history = np.array(sup_err_history)

In [ ]:
file_path = 'LSP_results_exact_fixed_basis_2d_no_con_basis_16.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=True)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

## num basis func = 25

In [55]:
basis_dim = 25
init_lr = .1

In [56]:
max_epochs = 500

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                                               diminish=False, 
                                                               intercept=False, weighted_avg=False, thresh_basis=thresh_basis,
                                                               record_fctn=get_sup_error_lsp_2d,
                                                               record_frequency=1, distribution='uniform',
                                                               opt_method='lbfgs',
                                                               device=device, trace_frequency=1)
sup_err_history = np.array(sup_err_history)

In [58]:
file_path = 'LSP_results_exact_fixed_basis_2d_no_con_basis_25.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=True)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,Unnamed: 0,num_itr,sup_err
0,0,20.0,0.019137
1,1,40.0,0.013508
2,2,60.0,0.005370
3,3,80.0,0.002461
4,4,100.0,0.001518
5,5,120.0,0.001041
6,6,140.0,0.000681
7,7,160.0,0.000370
8,8,180.0,0.000203
9,9,200.0,0.000175
